In [48]:
from genericpath import isdir
from posixpath import abspath
import re
import threading
import os
import boto3
from boto3.s3.transfer import TransferConfig
from botocore.utils import S3_ACCELERATE_WHITELIST


class Error(Exception):
    """Base class for exceptions in this module."""
    pass

def _is_s3(path:str)->bool:
    """
    Determines if the path is an s3 path
    """
    return path.startswith("s3://")

def _trim_path(path):
    """
    trims "/" at the end of path
    """
    if path.endswith("/"):
        path = path[:-1] # remove / at the end
    
    return path

def _is_dir(path: str)->bool:
    """
    Determines if a path is a directory
    """
    if _is_s3(path):
        return path.endswith("/")
    else:
        return os.path.isdir(os.path.abspath(path))

def _append_object(base_path, obj)->str:
    """
    Appends object to basepath
    """
    base_path = _trim_path(base_path)
    return f"{base_path}/{obj}"

def _extract_bucket_key(s3_uri: str)->tuple:
    """
    This extracts bucket name and key given s3 uri. 
    :param s3_uri: s3 uri of form s3://bucket_name/prefix1/prefix2/file.ext
    :return bucket, key tuple
    """
    s3_regex="^s3://([a-z0-9.-]+)/(.*)$"
    search =re.search(s3_regex, s3_uri)
    if search is None:
        raise Error("Invalid s3 uri: {}".format(s3_uri))
    return search.groups()

def _extract_immediate_prefix(obj_key:str)->str:
    """
    Extracts immediate prefix from source object
    """
    immed_prefix = ""
    if len(obj_key.split("/")) > 1:
        immed_prefix = obj_key.split("/")[-2]
    
    return immed_prefix

def _get_dest_obj_name(initial_src, obj):
    """
    Determines destination object name based on initial source from use input and listed object passed
    """
    immed_prefix = ""
    if _is_s3(initial_src):
        immed_prefix = _extract_immediate_prefix(_extract_bucket_key(initial_src)[1])
    else:
        if os.path.isdir(os.path.abspath(initial_src)):
            immed_prefix = os.path.basename(os.path.abspath(initial_src))
        else:
            immed_prefix = _extract_immediate_prefix(initial_src)
    
    if immed_prefix == "":
        return obj
    else:
        return obj.split("{}/".format(immed_prefix))[-1]

def _list_dir(dir_name:str)->list:
    """
    Lists files in a dir and it's sub directories recurssively
    """
    files_and_dirs = os.listdir(dir_name)
    list_of_files = []
    for file in files_and_dirs:
        completePath = os.path.join(dir_name, file)
        if os.path.isdir(completePath):
            list_of_files = list_of_files + _list_dir(completePath)
        else:
            list_of_files.append(completePath)

    return list_of_files

def _create_local_dir(file_path:str)->bool:
    """
    creates local directory if does not exist
    """
    try:
        directory = os.path.dirname(file_path)
        if directory == "":
            return True # nothing to create
        if not os.path.exists(directory):
            os.makedirs(directory)
    except Exception as exc:
        raise Error("Error {} occurred while creating local directory".format(exc))
    
    return True

def aws_s3_ls(s3_uri: str, list_extended=False)->list:
    """
    list s3 objects in a bucket/prefix
    :param s3_uri: s3 path to list
    :param list_extended: to list extedned details - owner, size, last modified and object name
    :return list of s3 objects
    """
    client = boto3.client("s3")
    bucket, prefix = _extract_bucket_key(s3_uri)
    s3_objects = []
    cont_token = None
    while (True):
        if cont_token is None:
            kwargs = {
                "Bucket": bucket,
                "MaxKeys": 100,
                "Prefix": prefix
            }
        else:
            kwargs = {
                "Bucket": bucket,
                "MaxKeys": 100,
                "Prefix": prefix,
                "ContinuationToken": cont_token
            }   
        try:
            response = client.list_objects_v2(**kwargs)
            if response["KeyCount"] == 0:
                print ("Requested s3 object doesn't exist.")
                break
            for record in response["Contents"]:
                if record["Size"] > 0: # ignore just prefix names
                    if list_extended:
                        s3_objects.append((record["Size"], 
                                           record["LastModified"].strftime("%Y%m%d %H:%M:%S.%s"), 
                                           record["Key"]))
                    else:
                        s3_objects.append(record["Key"])
            if response["IsTruncated"]:
                cont_token = response["NextContinuationToken"]
            else:
                break
        except Exception as exc:
            raise Error("Error {} occurred while listing objects.".format(exc))
    return s3_objects

def _list_objects(src: str)->list:
    """
    List objects based on file system
    """
    if _is_s3(src):
        return aws_s3_ls(src)
    else:
        if _is_dir(src):
            return _list_dir(src)
        else:
            return [src]

def _copy_s3_to_s3(src_bucket: str, src_key: str, dest_bucket: str, dest_key: str)->bool:
    """
    copies an s3 file to another s3 bucket
    """
    s3_resource = boto3.resource('s3')
    copy_source = {
        'Bucket': src_bucket,
        'Key': src_key
        }
    bucket = s3_resource.Bucket(dest_bucket)
    try:
        bucket.copy(copy_source, dest_key)
    except Exception as exc:
        raise Error("Error {} occurred while working with s3 object to s3 object.".format(exc))
    
    return True

def _copy_s3_to_local(src_bucket: str, src_key: str, dest: str)->bool:
    """
    copies an s3 file to local
    """
    s3_resource = boto3.resource('s3')
    try:
        s3_resource.Bucket(src_bucket).download_file(src_key, dest)
    except Exception as exc:
        raise Error("Error {} occurred while working on s3 object to local.".format(exc))
    
    return True

def _copy_local_to_s3(src: str, dest_bucket: str, dest_key: str)->bool:
    """
    copies local file to s3
    """
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(src, dest_bucket, dest_key)
    except Exception as exc:
        raise Error("Error {} occurred while working on local object to s3.".format(exc))
    
    return True

def _process_file_movement(src:str, dest:str, is_move=False)->bool:
    """
    copies/moves s3/local folder/file to s3/local
    """
    debug_str = "move" if (is_move) else "copy"
    
    objects = _list_objects(src) # list objects
    for obj in objects:
        if _is_dir(dest) or _is_dir(src):
            temp_dest = _append_object(dest, _get_dest_obj_name(src, obj))
        else:
            temp_dest = dest
        
        if _is_s3(src) and _is_s3(dest): #s3 to s3
            src_bucket, _ = _extract_bucket_key(src)
            dest_bucket, dest_key = _extract_bucket_key(temp_dest)
            print(f"{debug_str} file s3://{src_bucket}/{obj} to {temp_dest}")
            status = _copy_s3_to_s3(src_bucket, obj, dest_bucket, dest_key)
            if status and is_move:
                aws_s3_rm(f"s3://{src_bucket}/{obj}")
        elif _is_s3(src): # s3 to local
            src_bucket, _ = _extract_bucket_key(src)
            _create_local_dir(temp_dest) # create dir if doesn't exist
            print(f"{debug_str} file s3://{src_bucket}/{obj} to {temp_dest}")
            status = _copy_s3_to_local(src_bucket, obj, temp_dest)
            if status and is_move:
                aws_s3_rm(f"s3://{src_bucket}/{obj}")
        elif _is_s3(dest): # local to s3
            dest_bucket, dest_key = _extract_bucket_key(temp_dest)
            print(f"{debug_str} file {obj} to {temp_dest}")
            status = _copy_local_to_s3(obj, dest_bucket, dest_key)
            if status and is_move:
                os.remove(obj)       
        
        if not status:
            raise Error(f"S3 {debug_str} failed.")
    return True

def aws_s3_cp(src:str, dest:str)->bool:
    """
    Copies a local file or S3 object to another location locally or in S3

    :param src: source s3 object/prefix, local file/directory
    :param dest: dest s3 object/prefix, local file/directory
    :return True if copy is successful else False
    """
    if _is_s3(src) or _is_s3(dest):
        status = _process_file_movement(src, dest)
    else:
        raise Error("None of the src/dest is an s3 filesystem. Use local file utils to copy.")
    if not status:
            raise Error("S3 copy failed.")
    return True

def aws_s3_mv(src:str, dest:str)->bool:
    """
    Moves a local file or S3 object to another location locally or in S3

    :param src: source s3 object/prefix, local file/directory
    :param dest: dest s3 object/prefix, local file/directory
    :return True if copy is successful else False
    """
    if _is_s3(src) or _is_s3(dest):
        status = _process_file_movement(src, dest, True)
    else:
        raise Error("None of the src/dest is an s3 filesystem. Use local file utils to move.")
    if not status:
            raise Error("S3 move failed.")
    return True

def aws_s3_rm(s3_key: str)->bool:
    """
    Delete s3 object or multiple objects from an s3 bucket.
    :param s3_key: s3 key in the form "s3://bucket_name/file_name.txt", if the s3 path ends with "/" entire prefix will be deleted
    :return True if delete is successful else False
    """
    client = boto3.client("s3")
    #list objects
    objs_to_delete = aws_s3_ls(s3_key)
    if len(objs_to_delete) ==0: # no files to delete
        return True
    s3_bucket, _ = _extract_bucket_key(s3_key)
    #prepare s3 objects to delete
    objs_list=[]
    for obj in objs_to_delete:
        objs_list.append({"Key": obj})
    
    try:
        response = client.delete_objects(
            Bucket=s3_bucket,
            Delete={
                'Objects': objs_list,
                'Quiet': False
            }
        )
    except Exception as exc:
        raise Error("Cannot delete, exception {} occurred".format(exc))
    
    return True

In [4]:
src="s3://mufg-data-sandbox-us-east-1-sagemaker/awsLogs/"
dest="s3://mufg-data-sandbox-us-east-1-sagemaker/AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs"

### List files in s3

In [2]:
aws_s3_ls("s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh")

['sagemaker/bootstrap/scripts/add_mand_tags.sh']

In [5]:
aws_s3_ls(src)

['awsLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/01b276ae-c222-37c5-bfeb-18d88ee71008.jsonl.gz',
 'awsLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/8f7774cb-1059-3831-a14f-7993126b0a4e.jsonl.gz',
 'awsLogs/Macie/us-west-2/649ba993d1ebb03a5cc3da2e2f2b516d/476054568469/7c2379e2-3e6e-34e5-a296-d55ca84e193f.jsonl.gz',
 'awsLogs/Macie/us-west-2/6e32127d5763fdff30e17627a1746551/476054568469/0ac29785-c308-38c0-b2b7-9bb2338bdcb1.jsonl.gz',
 'awsLogs/Macie/us-west-2/6e32127d5763fdff30e17627a1746551/476054568469/6c181bb1-096a-34cd-99f3-36dfc90cc7ec.jsonl.gz',
 'awsLogs/Macie/us-west-2/96f0d21cb4f77b3e28f6e370d0411e98/476054568469/80fa7a49-0d2b-33a0-9d33-50b18a73f1e3.jsonl.gz',
 'awsLogs/Macie/us-west-2/96f0d21cb4f77b3e28f6e370d0411e98/476054568469/ac5d8475-8b09-307a-9e72-bbeb8622922b.jsonl.gz',
 'awsLogs/Macie/us-west-2/96f0d21cb4f77b3e28f6e370d0411e98/476054568469/d9eff6c1-eacc-3c6e-bfe7-ea0e7234e3be.jsonl.gz',
 'awsLogs/Macie/us-west-2/96f0d21cb4f77b

### copy files from s3 to s3

In [11]:
aws_s3_cp(src, dest)

copy file s3://mufg-data-sandbox-us-east-1-sagemaker/awsLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/01b276ae-c222-37c5-bfeb-18d88ee71008.jsonl.gz to s3://mufg-data-sandbox-us-east-1-sagemaker/AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/01b276ae-c222-37c5-bfeb-18d88ee71008.jsonl.gz
copy file s3://mufg-data-sandbox-us-east-1-sagemaker/awsLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/8f7774cb-1059-3831-a14f-7993126b0a4e.jsonl.gz to s3://mufg-data-sandbox-us-east-1-sagemaker/AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/8f7774cb-1059-3831-a14f-7993126b0a4e.jsonl.gz
copy file s3://mufg-data-sandbox-us-east-1-sagemaker/awsLogs/Macie/us-west-2/649ba993d1ebb03a5cc3da2e2f2b516d/476054568469/7c2379e2-3e6e-34e5-a296-d55ca84e193f.jsonl.gz to s3://mufg-data-sandbox-us-east-1-sagemaker/AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/649

True

In [12]:
aws_s3_ls(dest)

['AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/01b276ae-c222-37c5-bfeb-18d88ee71008.jsonl.gz',
 'AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/8f7774cb-1059-3831-a14f-7993126b0a4e.jsonl.gz',
 'AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/649ba993d1ebb03a5cc3da2e2f2b516d/476054568469/7c2379e2-3e6e-34e5-a296-d55ca84e193f.jsonl.gz',
 'AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/6e32127d5763fdff30e17627a1746551/476054568469/0ac29785-c308-38c0-b2b7-9bb2338bdcb1.jsonl.gz',
 'AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/6e32127d5763fdff30e17627a1746551/476054568469/6c181bb1-096a-34cd-99f3-36dfc90cc7ec.jsonl.gz',
 'AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/96f0d21cb4f77b3e28f6e370d0411e98/476054568469/80fa7a49-0d2b-33a0-9d33-50b18a73f1e3.jsonl.gz',
 'AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/96f0d21cb4f77b3e28f6e370d0411e

### delete file/dir from s3

In [14]:
aws_s3_rm("s3://mufg-data-sandbox-us-east-1-sagemaker/AROAW5VYQHYKT53GIKCDT:mthummati/AWSLogs/Macie/us-west-2/ad69687a856ed92b6d7f08a2630cf7df/476054568469/f85e8081-addf-387a-af8d-cd087c89b1fa.jsonl.gz")

True

In [15]:
aws_s3_rm(dest)

True

In [16]:
aws_s3_ls(dest)

Requested s3 object doesn't exist.


[]

### download file/dir to local

In [19]:
aws_s3_cp("s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh", "test.txt")

copy file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh to test.txt


True

In [21]:
aws_s3_cp("s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh", "/home/ec2-user/SageMaker")

copy file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh to /home/ec2-user/SageMaker/add_mand_tags.sh


True

In [24]:
aws_s3_cp("s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/", "/home/ec2-user/SageMaker/bootstrap/")

copy file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/add_mand_tags.sh to /home/ec2-user/SageMaker/bootstrap/add_mand_tags.sh
copy file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh to /home/ec2-user/SageMaker/bootstrap/scripts/add_mand_tags.sh
copy file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_pip_repo.sh to /home/ec2-user/SageMaker/bootstrap/scripts/add_pip_repo.sh
copy file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/bootstrap_sagemaker.sh to /home/ec2-user/SageMaker/bootstrap/scripts/bootstrap_sagemaker.sh
copy file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/configure_git.sh to /home/ec2-user/SageMaker/bootstrap/scripts/configure_git.sh
copy file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/test.file to /home/ec2-user/SageMaker/bootstrap/test.file


True

### upload file to s3

In [39]:
aws_s3_cp("/home/ec2-user/SageMaker/scripts/", "s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/")

copy file /home/ec2-user/SageMaker/scripts/add_pip_repo.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_pip_repo.sh
copy file /home/ec2-user/SageMaker/scripts/configure_git.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/configure_git.sh
copy file /home/ec2-user/SageMaker/scripts/bootstrap_sagemaker.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/bootstrap_sagemaker.sh
copy file /home/ec2-user/SageMaker/scripts/add_mand_tags.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh


True

In [50]:
aws_s3_cp("/home/ec2-user/SageMaker/scripts/add_mand_tags.sh", "s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/test/")

copy file /home/ec2-user/SageMaker/scripts/add_mand_tags.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/test/add_mand_tags.sh


True

In [52]:
aws_s3_cp("add_mand_tags.sh", "s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/test/")

copy file add_mand_tags.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/test/add_mand_tags.sh


True

In [51]:
aws_s3_cp("/home/ec2-user/SageMaker/scripts/add_mand_tags.sh", "s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/test.file")

copy file /home/ec2-user/SageMaker/scripts/add_mand_tags.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/test.file


True

### Move files from s3 to s3

In [54]:
src="s3://mufg-data-sandbox-us-east-1-sagemaker/AWSLogs/"
dest="s3://mufg-data-sandbox-us-east-1-sagemaker/awsLogs/"
aws_s3_mv(dest, src)

move file s3://mufg-data-sandbox-us-east-1-sagemaker/awsLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/01b276ae-c222-37c5-bfeb-18d88ee71008.jsonl.gz to s3://mufg-data-sandbox-us-east-1-sagemaker/AWSLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/01b276ae-c222-37c5-bfeb-18d88ee71008.jsonl.gz
move file s3://mufg-data-sandbox-us-east-1-sagemaker/awsLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/8f7774cb-1059-3831-a14f-7993126b0a4e.jsonl.gz to s3://mufg-data-sandbox-us-east-1-sagemaker/AWSLogs/Macie/us-west-2/348da50ddca9309ae4214ccdff2fa5b3/476054568469/8f7774cb-1059-3831-a14f-7993126b0a4e.jsonl.gz
move file s3://mufg-data-sandbox-us-east-1-sagemaker/awsLogs/Macie/us-west-2/649ba993d1ebb03a5cc3da2e2f2b516d/476054568469/7c2379e2-3e6e-34e5-a296-d55ca84e193f.jsonl.gz to s3://mufg-data-sandbox-us-east-1-sagemaker/AWSLogs/Macie/us-west-2/649ba993d1ebb03a5cc3da2e2f2b516d/476054568469/7c2379e2-3e6e-34e5-a296-d55ca84e193f.jsonl.gz
move fi

True

In [58]:
aws_s3_mv("s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh", "/home/ec2-user/SageMaker/")

move file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh to /home/ec2-user/SageMaker/add_mand_tags.sh


True

In [59]:
aws_s3_mv("/home/ec2-user/SageMaker/add_mand_tags.sh", "s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/")

move file /home/ec2-user/SageMaker/add_mand_tags.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh


True

In [65]:
aws_s3_mv("test", "s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts")

move file test/add_pip_repo.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_pip_repo.sh
move file test/configure_git.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/configure_git.sh
move file test/bootstrap_sagemaker.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/bootstrap_sagemaker.sh
move file test/add_mand_tags.sh to s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh


True

In [66]:
aws_s3_mv("s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/", "/home/ec2-user/SageMaker/test")

move file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_mand_tags.sh to /home/ec2-user/SageMaker/test/add_mand_tags.sh
move file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/add_pip_repo.sh to /home/ec2-user/SageMaker/test/add_pip_repo.sh
move file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/bootstrap_sagemaker.sh to /home/ec2-user/SageMaker/test/bootstrap_sagemaker.sh
move file s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/configure_git.sh to /home/ec2-user/SageMaker/test/configure_git.sh


True

In [63]:
aws_s3_ls("s3://mufg-data-sandbox-us-east-1-sagemaker/sagemaker/bootstrap/scripts/")

Requested s3 object doesn't exist.


[]

In [34]:
util._get_dest_obj_name("test", "test/file.txt")


'file.txt'

In [52]:
import os
os.path.basename(os.path.abspath("dest/test.file"))
#os.path.isdir(os.path.abspath("dest"))

'test.file'

In [38]:
util._extract_immediate_prefix("test/file.txt/")

'file.txt'